# Clase sobre estadistica para DataScience
### Temporalidad

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

# Crear una serie temporal simulada con tendencia y estacionalidad
np.random.seed(42)
periodos = 120  # 10 años de datos mensuales
tiempo = pd.date_range(start='2015-01', periods=periodos, freq='ME')

# Componentes de la serie
tendencia = np.linspace(10, 50, periodos)                          # crecimiento lineal
estacionalidad = 10 * np.sin(2 * np.pi * tiempo.month / 12)       # estacionalidad anual
ruido = np.random.normal(0, 2, periodos)                          # componente aleatoria

# Serie final
serie = tendencia + estacionalidad + ruido
df = pd.DataFrame({'Fecha': tiempo, 'Valor': serie}).set_index('Fecha')

# Graficar
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['Valor'], label='Serie Temporal')
plt.title('Ejemplo de Serie Temporal con Tendencia y Estacionalidad')
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
df_trimestral = df.resample('QE').mean().sort_index()  # También puedes usar .sum(), .median(), etc.
df_trimestral.head()

In [ ]:

# Graficar la serie trimestral
plt.figure(figsize=(12, 5))
plt.plot(df_trimestral.index, df_trimestral['Valor'], marker='o', color='orange', label='Valor Trimestral (promedio)')
plt.title('Serie Temporal Trimestral')
plt.xlabel('Fecha')
plt.ylabel('Valor Promedio Trimestral')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Agregar columnas de año y trimestre
df['Año'] = df.index.year
df['Trimestre'] = df.index.quarter

# Agrupar por año y trimestre y calcular estadísticas
resumen_trimestral = df.groupby(['Año', 'Trimestre'])['Valor'].agg(['mean', 'std', 'min', 'max']).reset_index()

print(resumen_trimestral.head())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Paleta de 4 colores distintos de Seaborn
colores = sns.color_palette('tab10', 4)

fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)

for i, trimestre in enumerate(range(1, 5)):
    ax = axes[i//2, i%2]
    datos = df[df['Trimestre'] == trimestre]
    
    # Agrupar para el promedio anual dentro del trimestre
    resumen = datos.groupby('Año')['Valor'].mean().reset_index()
    
    # Graficar con color distinto por trimestre
    ax.plot(resumen['Año'], resumen['Valor'], marker='o', color=colores[i], label=f'Trimestre {trimestre}')
    ax.set_title(f'Evolución Valor Promedio - Trimestre {trimestre}')
    ax.set_xlabel('Año')
    ax.set_ylabel('Valor Promedio')
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns


In [ ]:

# Gráfico boxplot de valores por trimestre
plt.figure(figsize=(10, 6))
sns.boxplot(x='Trimestre', y='Valor', data=df)
plt.title('Distribución Trimestral de la Serie')
plt.xlabel('Trimestre')
plt.ylabel('Valor')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
for trimestre in range(1, 5):
    datos = resumen_trimestral[resumen_trimestral['Trimestre'] == trimestre]
    plt.plot(datos['Año'], datos['mean'], marker='o', label=f'Trimestre {trimestre}')

plt.title('Evolución del Promedio por Trimestre')
plt.xlabel('Año')
plt.ylabel('Valor Promedio')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### 1. **¿Hay trimestres que crecen más rápido?**

Si mirás el gráfico de la evolución del promedio por trimestre (último gráfico), vas a ver una línea para cada trimestre a lo largo de los años.

* Como la serie tiene una **tendencia lineal creciente** (de 10 a 50 en 10 años), todas las líneas deberían mostrar una tendencia creciente.
* Sin embargo, algunos trimestres pueden tener una pendiente más pronunciada si la estacionalidad o el ruido afectan más en esos períodos.

Dado que la estacionalidad es una sinusoide anual (12 meses), los trimestres que incluyen meses con mayor valor en la función seno tendrán promedios trimestrales más altos, pero la pendiente de crecimiento debe ser parecida entre todos.

---

### 2. **¿Hay trimestres sistemáticamente más altos o bajos (Estacionalidad)?**

Sí, la estacionalidad anual definida como:

```python
estacionalidad = 10 * np.sin(2 * np.pi * tiempo.month / 12)
```

significa que ciertos meses del año tendrán valores más altos o bajos.

* El trimestre que incluye los meses con mayor valor del seno tendrá promedios trimestrales más altos.
* Por ejemplo, meses alrededor de marzo o septiembre tendrán picos o valles.

En el boxplot de distribución trimestral, vas a ver que algunos trimestres tienen la mediana y rango de valores consistentemente más altos o bajos, lo que refleja esa estacionalidad.

---

### 3. **¿Qué tan volátil es cada trimestre (Variabilidad)?**

En el boxplot podés observar la dispersión de los datos por trimestre:

* Si un trimestre tiene cajas más anchas y bigotes más largos, indica mayor variabilidad (ruido o fluctuaciones mayores).
* En esta simulación, el ruido es aleatorio con distribución normal y desviación estándar constante (±2), entonces la variabilidad debería ser similar entre trimestres.
* Sin embargo, la combinación de tendencia y estacionalidad puede afectar ligeramente la variabilidad percibida.

---

### 4. **¿Hay anomalías o picos inesperados por trimestre?**

* En esta serie simulada, el ruido es aleatorio pero con distribución normal, sin valores extremos muy marcados.
* Por eso, es poco probable que haya anomalías o outliers claros.
* En el boxplot, podés ver puntos atípicos si existen valores alejados del rango intercuartílico.

---

### Resumen:

| Pregunta                          | Respuesta esperada en tu serie simulada                                        |
| --------------------------------- | ------------------------------------------------------------------------------ |
| Trimestres que crecen más rápido  | Todos crecen similarmente por la tendencia lineal, no hay grandes diferencias. |
| Trimestres sistemáticamente altos | Sí, reflejan la estacionalidad definida (por meses dentro del trimestre).      |
| Volatilidad de cada trimestre     | Similar entre trimestres debido a ruido con varianza constante.                |
| Anomalías o picos inesperados     | No, la serie es simulada con ruido normal sin outliers fuertes.                |

